In [1]:
import pandas as pd

In [2]:
import pyTigerGraph as tg
conn = tg.TigerGraphConnection(host="https://mytg.i.tgcloud.io", password="pass123")
conn.graphname = "Mask"
conn.apiToken = conn.getToken(conn.createSecret())

In [9]:
print(conn.gsql('''
CREATE QUERY st(/* Parameters here */) FOR GRAPH Mask { 
  /* Write query logic here */
  Seed = {test.*};
  tmp = SELECT s From Seed:s;
  PRINT tmp; 
}
INSTALL QUERY st
''', options=[]))

Successfully created queries: [st].
Start installing queries, about 1 minute ...
st query: curl -X GET 'https://127.0.0.1:9000/query/Mask/st'. Add -H "Authorization: Bearer TOKEN" if authentication is enabled.
Select 'm1' as compile server, now connecting ...
Node 'm1' is prepared as compile server.

Query installation finished.


In [35]:
select = conn.runInstalledQuery("st", {})

In [11]:
select

[{'tmp': [{'v_id': '1112',
    'v_type': 'test',
    'attributes': {'id': 1112,
     'name': 'polip',
     'email': 'polip.m2@gmail.com',
     'mobile': '082164510298',
     'credit': '52310259 1269102392'}},
   {'v_id': '1111',
    'v_type': 'test',
    'attributes': {'id': 1111,
     'name': 'Maul',
     'email': 'maul@gmail.com',
     'mobile': '087808235768',
     'credit': '1234 4214 1521 5132'}}]}]

In [24]:
import flat_table
df_tx_hop = pd.DataFrame(select)
df_tx_hop = flat_table.normalize(df_tx_hop)
df_tx_hop = df_tx_hop.drop(columns=["index", "tmp.v_type", "tmp.v_id"])

In [25]:
df_tx_hop

,tmp.attributes.credit,tmp.attributes.mobile,tmp.attributes.email,tmp.attributes.name,tmp.attributes.id
0,52310259 1269102392,082164510298,polip.m2@gmail.com,polip,1112
1,1234 4214 1521 5132,087808235768,maul@gmail.com,Maul,1111


In [26]:
df_tx_hop.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 5 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   tmp.attributes.credit  2 non-null      object
 1   tmp.attributes.mobile  2 non-null      object
 2   tmp.attributes.email   2 non-null      object
 3   tmp.attributes.name    2 non-null      object
 4   tmp.attributes.id      2 non-null      object
dtypes: object(5)
memory usage: 208.0+ bytes


In [28]:
df_tx_hop['tmp.attributes.id'] = df_tx_hop['tmp.attributes.id'].astype(int)

In [29]:
df_tx_hop.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 5 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   tmp.attributes.credit  2 non-null      object
 1   tmp.attributes.mobile  2 non-null      object
 2   tmp.attributes.email   2 non-null      object
 3   tmp.attributes.name    2 non-null      object
 4   tmp.attributes.id      2 non-null      int32 
dtypes: int32(1), object(4)
memory usage: 200.0+ bytes


In [30]:
def mask_email(Var):
    mail=Var.split("@")[0]
    n=len(mail)
    alist=list(mail)
    alist[1:int(n)-1]='*'*int(n-2)
    output="".join(alist)+"@"+Var.split("@")[1]
    return output
def mask_mobile(Var):
    Var=str(Var)
    n=len(Var)
    alist=list(Var)
    alist[2:int(n)-2]='*'*int(n-4)
    output="".join(alist)
    return output
import re
def mask_credit(Var):
    output = re.sub('\d', '*', Var[:-4]) + Var[-4:]
    return output

In [33]:
df_tx_hop['tmp.attributes.email']= df_tx_hop['tmp.attributes.email'].apply(mask_email)
df_tx_hop['tmp.attributes.mobile']= df_tx_hop['tmp.attributes.mobile'].apply(mask_mobile)
df_tx_hop['tmp.attributes.credit']= df_tx_hop['tmp.attributes.credit'].apply(mask_credit)
df_tx_hop

,tmp.attributes.credit,tmp.attributes.mobile,tmp.attributes.email,tmp.attributes.name,tmp.attributes.id
0,******** ******2392,08********98,p******2@gmail.com,polip,1112
1,**** **** **** 5132,08********68,m**l@gmail.com,Maul,1111


In [34]:
v_test = conn.upsertVertexDataFrame(df_tx_hop, "test", "tmp.attributes.id", attributes={"id": "tmp.attributes.id", "name": "tmp.attributes.name", "email": "tmp.attributes.email", "mobile": "tmp.attributes.mobile", "credit": "tmp.attributes.credit"})

In [ ]:
df_tx_hop.to

In [ ]:
import json
#Load the posts file with the 'load_people' job
people_file='mask_data.csv'
results=conn.uploadFile(people_file, fileTag='MyDataSource', jobName='load_people')
print(json.dumps(results,indent=2))